# DatasetMake

Here, I endeavor to assemble a suitable testing document. This testing document will be a random selection of documents from our corpus, but it will not be a _uniform_ random selection. Unique pages will be enriched by a known, quantifiable amount. The intent of selecting pages that are observed to be "unique" is to:
1. Represent as many _languages_ as possible that feature reasonably prominently in the dataset.
    * No language (e.g., all pictures) is one possibility here.
1. Represent all _page orientations_ that feature reasonably prominently in the dataset, possibly including:
    * 0 degrees
    * 90 degrees
    * 180 degrees
    * 270 degrees
    * None of the above
1. Represent all exceptional elements that feature reasonably prominently in the dataset, possibly including:
    * Presence of at least one image
    * Presence of tabular data

In this case, "reasonably prominently" means "with high enough frequency as to have a non-negligible effect on overall OCR accuracy."

I also endeavor to quantify the frequency of these characteristics for the following reasons:
* It is necessary to determine which circumstances feature "reasonably prominently."
* More importantly, it is necessary to determine the magnitude of the effect of any bug on our OCR output. This will make our accuracy metrics more meaningful.

Here, I handle imports:

In [14]:
#!pip install PyPDF2
!pip install PyMuPDF
!pip install -U -q PyDrive

     |████████████████████████████████| 6.4MB 8.4MB/s 


Here, initialize drive access:

In [2]:
from google.colab import drive
drive.mount('/content/drive')

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from pydrive.settings import InvalidConfigError
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Here, I import needed packages:

In [15]:
#from PyPDF2 import PdfFileReader, PdfFileWriter, PdfFileMerger
#from PyPDF2.utils import PdfReadError
import fitz
from shutil import copyfile
import os
import numpy as np
import pandas as pd
import time
import gspread
from oauth2client.client import GoogleCredentials
from google.colab import auth

Here, I declare global variables and utility functions:

In [4]:
CATALOG_NAMES = [
  'catalog.20200407.aa', 'catalog.20200407.ab', 'catalog.20200407.ac',
  'catalog.20200407.ad', 'catalog.20200407.ae'
]
LARGE_SAMPLE_PATH = ('/content/drive/MyDrive/AWCA/Colab_notebooks/OCR/pDonovan/'
                     'awca-ocr/large_sample.pdf')
COMPRESSED_SAMPLE_PATH = ('/content/drive/MyDrive/AWCA/Colab_notebooks/OCR/'
                          'pDonovan/awca-ocr/compressed_sample.pdf')
CORPUS_ROOT = '/content/drive/MyDrive/AWCA/PDFtp'
WORKING_DIR = ('/content/drive/MyDrive/AWCA/Colab_notebooks/OCR/'
               'pDonovan/awca-ocr/temp')

In [5]:
def get_df(title, gc, worksheet=0, has_headers=True):
    """Returns a pandas.DataFrame representation of the
    (WORKSHEET)th worksheet of the Google Sheets (GSHEET)
    file that has title TITLE.
    TITLE - the title of the desired spreadsheet
    GC    - the GSpread credentials needed to retrieve the spreadsheet
    WORKSHEET - the index of the desired worksheet within
        the spreadsheet
    HAS_HEADERS - set to False if the spreadsheet does not
        have a header row at the top.
    It is not necessary to specify the path or the GSHEET
    file extension. Note that this creates undefined
    behavior when your google drive has multiple spreadsheets
    with the same name (i.e., you do not know which one
    will be opened).
    """
    # For details on how to handle GSHEET files, see
    # https://gspread.readthedocs.io/en/latest/api.html
    contents = gc.open(title).get_worksheet(worksheet).get_all_values()
    if has_headers:
        return pd.DataFrame.from_records(
            data=contents[1:],
            columns=contents[0]
        )
    return pd.DataFrame.from_records(contents)

In [6]:
auth.authenticate_user()
GC = gspread.authorize(GoogleCredentials.get_application_default())

In [7]:
CATALOG = pd.concat(
    (get_df(name, GC) for name in CATALOG_NAMES),
    ignore_index=True
)
CATALOG.head()

,ID,md5,Size,mime-type,Created Date,Modified Date,Folder,Name,gdoc-id,gdoc-url,gdoc-length,gdoc-exceptions,gdoc-timestamp,delta-t (s),Notes
0,0BwjSAKD6JzR6WXBVczdtRVBuVG8,c2bbbcdb46850c82a6f0639122da65b3,43036522,video/quicktime,2017-03-06T21:54:39.618Z,2017-03-06T21:54:39.618Z,ane.pdf.share/By Topic (or field)/Teaching Tools,Flex Search Workspace.mov,,,,,,,
1,0ByFFNduW4doJWUhIQmhqQ0ZCbnM,4b182737a83195784fccbbd2c2532e5a,11711124,video/quicktime,2013-01-31T00:48:04.693Z,2013-01-31T00:48:04.693Z,ane.pdf.share/By Topic (or field)/Teaching Too...,V22i3003 -ls T718 dM ls rain movie.MOV,,,,,,,
2,0ByFFNduW4doJSlZYZTdCTG1oa0E,4b182737a83195784fccbbd2c2532e5a,11711124,video/quicktime,2013-01-31T00:48:46.458Z,2013-01-31T00:48:46.458Z,ane.pdf.share/By Topic (or field)/Teaching Too...,V22i3003 -ls T718 dM rain movie.MOV,,,,,,,
3,0ByFFNduW4doJWThDMC15Nm1Ucms,0b760c00c95eb567b91f195a6f6badba,11831672,video/quicktime,2013-01-31T00:49:32.192Z,2013-01-31T00:49:32.192Z,ane.pdf.share/By Topic (or field)/Teaching Too...,V22i3004 -ls T718 dM draining movie.MOV,,,,,,,
4,0ByFFNduW4doJT0lDUFdSeEdRdkU,0b760c00c95eb567b91f195a6f6badba,11831672,video/quicktime,2013-01-31T00:50:18.413Z,2013-01-31T00:50:18.413Z,ane.pdf.share/By Topic (or field)/Teaching Too...,V22i3004 -ls T718 dM ls draining movie.MOV,,,,,,,


In [8]:
CATALOG.shape

(530285, 15)

In [9]:
CATALOG.sample(5)

,ID,md5,Size,mime-type,Created Date,Modified Date,Folder,Name,gdoc-id,gdoc-url,gdoc-length,gdoc-exceptions,gdoc-timestamp,delta-t (s),Notes
447832,1a4-N2ZvdLt8rifeJiofi3Lh3cBRxIne0,bbe7fceff80c3cef202a3e0b95072691,4096,application/pdf,2019-07-21T19:57:21.259Z,2019-07-21T19:57:21.259Z,ane.pdf.share/To be sorted.../EDUB2BAA/Strahil...,._Tadmor - 1958 (JNES 17) - dâku.pdf,NaN,NaN,NaN,NaN,NaN,NaN,NaN
325064,11T3axPel0nZHz0cl-6oRmaGNB76fwJhr,e8f853891669e605350e1925005d9298,625557,application/pdf,2019-07-02T09:06:02.370Z,2019-07-02T09:06:02.370Z,ane.pdf.share/To be sorted.../EDUB2BAA/C Anato...,4 Review of Excavations at Tepe Gawra volume I...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
462357,16Vogm4-awQ80zyns-N_OkKTx9UEhc1Gw,dff1ab7e624ed29ef901d54c376fb226,159911,image/jpeg,2019-07-22T05:13:02.896Z,2019-07-22T05:13:02.896Z,ane.pdf.share/To be sorted.../EDUB2BAA/Strahil...,S1445.jpg,NaN,NaN,NaN,NaN,NaN,NaN,NaN
304048,1ku_FlVb3ZAMlJFLuCRDfHB1yfiktKJiX,3b58e575402eab28593a32feb2f32ad3,188383,application/pdf,2019-07-03T07:58:32.613Z,2019-07-03T07:58:32.613Z,ane.pdf.share/To be sorted.../EDUB2BAA/Bit Enk...,"Iraq 25, Kupper, Opinion publique Mari.pdf",NaN,NaN,NaN,NaN,NaN,NaN,NaN
506432,1zdPPCyTaqeJyJSYjTVJLT16ard6gVtgN,1,1,application/vnd.google-apps.folder,2019-08-16T22:57:24.383Z,2019-08-16T22:57:24.383Z,ane.pdf.share/To be sorted.../EDUB2BAA/Willis/...,,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## 1. Build a Large Representative Document

The objective here is simply to take a uniform random sample of pages and to stitch them all into one PDF for easy reference.

**Note:** I am aware that the following two methods are hacks. They are ugly, and they use weird workarounds. The only reason why I am leaving this code this way is that I only intend to run this code ONCE. If anyone else ever decides they need this code, it is advisable to REWRITE IT, not to patch it or maintain it.

In [10]:
def sample_pdfs(catalog, n, root, working_dir, random):
  """Returns an iterable of N paths to copies of PDF files selected
  uniformly at random from CATALOG.
  """
  pdfs = catalog[
    (catalog['mime-type'] == 'application/pdf')
    & np.array([name[:2] != '._' for name in catalog.Name])
  ]
  # The exact value 1e9 is unimportant -- it is not a magic number.
  pdfs = pdfs.sample(n, random_state=random.integers(1e9), replace=True)
  pdfs.index = list(range(len(pdfs.index)))
  original_paths = pdfs.apply(
      (lambda row: os.path.join(root, row.Folder, row.Name)),
      axis=1,
  )
  new_paths = pdfs.apply(
      (lambda row: (os.path.join(working_dir, row.ID) + '.pdf')),
      axis=1
  )
  paths = list()
  for idx in original_paths.index:
    while True:
      try:
        print('DEBUG: ', idx, ', ', new_paths[idx])
        if os.path.exists(new_paths[idx]):
          print('DEBUG: found PDF at ', new_paths[idx])
          paths.append(new_paths[idx])
        elif os.path.exists(original_paths[idx]):
          print('DEBUG: found PDF at ', original_paths[idx])
          paths.append(original_paths[idx])
        else:
          drive.CreateFile({'id': pdfs.loc[idx, 'ID']}).GetContentFile(new_paths[idx])
          print('DEBUG: created PDF at ', new_paths[idx])
          paths.append(new_paths[idx])
        break
      except Exception as e:
        print('DEBUG: ', e)
        auth.authenticate_user()
        GoogleAuth()
        gauth.credentials = GoogleCredentials.get_application_default()
        GoogleDrive(gauth)
  return paths

In [55]:
def combine(pdfs, output_path, random):
  stub = os.path.splitext(output_path)[0]
  output_paths = (stub + '(0).pdf', stub + '(1).pdf')
  for path in output_paths:
    if not os.path.exists(path):
      copyfile(output_path, path)
  output_lengths = tuple(
      PdfFileReader(path).getNumPages() for path in output_paths)
  errors_count = 0
  i = 1 # This is the number of pages in the stub.
  for pdf in pdfs:
    if i >= output_lengths[i % 2]:
      input0 = output_paths[(i + 1) % 2]
      input1 = pdf
      output = output_paths[i % 2]
      try:
        merger = PdfFileMerger()
        merger.append(input0)
        reader = PdfFileReader(input1)
        chosen = random.integers(reader.getNumPages())
        merger.append(reader, pages=(chosen, chosen+1))
        merger.write(output)
        merger.close()
      except Exception as e:
        print(e)
        print('Failed at file:', pdfs[i])
        errors_count += 1
    i += 1
  print('Out of {} documents, {} could not be read ({:.4f}%).'.format(
            len(pdfs), errors_count, errors_count / len(pdfs) * 100,
        ))
  os.rename(output_paths[i % 2], output_path)
def combine(pdfs, output_path, random):
  out = fitz.open()
  pdfs = list(pdfs)
  errors = list()
  for i, path in enumerate(pdfs):
    try:
      pdf = fitz.open(path, filetype='pdf')
      assert len(pdf) > 0, 'PDF must have at least one page.'
    except Exception as e:
      errors.append(str(e))
      print(e)
      continue
    chosen = int(random.integers(len(pdf)))
    print('{} DEBUG: Selecting page {} from {}'.format(i, chosen, path))
    out.insert_pdf(pdf, from_page=chosen, to_page=chosen)
  print('Terminated with {} errors.'.format(errors))
  out.save(output_path)
  return errors

In [56]:
random = np.random.default_rng(333) # Time of day when this code was written
combine(
    sample_pdfs(CATALOG, 1600, CORPUS_ROOT, WORKING_DIR, random),
    LARGE_SAMPLE_PATH, random
)

DEBUG:  0 ,  /content/drive/MyDrive/AWCA/Colab_notebooks/OCR/pDonovan/awca-ocr/temp/1T5zmjoa2zcBkiIu6vbpoxcGmoFmS7ZqO.pdf
DEBUG: found PDF at  /content/drive/MyDrive/AWCA/Colab_notebooks/OCR/pDonovan/awca-ocr/temp/1T5zmjoa2zcBkiIu6vbpoxcGmoFmS7ZqO.pdf
DEBUG:  1 ,  /content/drive/MyDrive/AWCA/Colab_notebooks/OCR/pDonovan/awca-ocr/temp/1CpervdsqrDmIOuEprs2woDBKiIgTwCDi.pdf
DEBUG: found PDF at  /content/drive/MyDrive/AWCA/Colab_notebooks/OCR/pDonovan/awca-ocr/temp/1CpervdsqrDmIOuEprs2woDBKiIgTwCDi.pdf
DEBUG:  2 ,  /content/drive/MyDrive/AWCA/Colab_notebooks/OCR/pDonovan/awca-ocr/temp/1MHzaehYGC-RwqCfrSIrB3cnQo6Guuy2c.pdf
DEBUG: found PDF at  /content/drive/MyDrive/AWCA/Colab_notebooks/OCR/pDonovan/awca-ocr/temp/1MHzaehYGC-RwqCfrSIrB3cnQo6Guuy2c.pdf
DEBUG:  3 ,  /content/drive/MyDrive/AWCA/Colab_notebooks/OCR/pDonovan/awca-ocr/temp/0ByFFNduW4doJZFdBRkRTa0dlM3c.pdf
DEBUG: found PDF at  /content/drive/MyDrive/AWCA/PDFtp/ane.pdf.share/By Topic (or field)/Univ Presses/Oxford Univ Press/0195393

mupdf: expected trailer marker
mupdf: invalid key in dict
mupdf: invalid key in dict
mupdf: invalid key in dict
mupdf: invalid key in dict
mupdf: invalid key in dict
mupdf: invalid key in dict
mupdf: invalid key in dict
mupdf: invalid key in dict
mupdf: invalid key in dict


106 DEBUG: Selecting page 6 from /content/drive/MyDrive/AWCA/Colab_notebooks/OCR/pDonovan/awca-ocr/temp/1RYJaBUAb5xS5t7VDjiXdK4rmaqqiy-zR.pdf
107 DEBUG: Selecting page 240 from /content/drive/MyDrive/AWCA/Colab_notebooks/OCR/pDonovan/awca-ocr/temp/1BbERM8OTfrKsB4aAsUgGHFb0pbb1AQ1d.pdf
108 DEBUG: Selecting page 10 from /content/drive/MyDrive/AWCA/Colab_notebooks/OCR/pDonovan/awca-ocr/temp/1ur6vuunbBT474-YexKMBq_V3TmvNTjv4.pdf
109 DEBUG: Selecting page 487 from /content/drive/MyDrive/AWCA/PDFtp/ane.pdf.share/By Author (or editor)/B/B/Bright, John - A History of Israel, 4th Edition. 2000.pdf
110 DEBUG: Selecting page 0 from /content/drive/MyDrive/AWCA/Colab_notebooks/OCR/pDonovan/awca-ocr/temp/1XuQiwz_S_nikQcC2DkuB6uvTAlEEQWll.pdf
111 DEBUG: Selecting page 2 from /content/drive/MyDrive/AWCA/Colab_notebooks/OCR/pDonovan/awca-ocr/temp/1ZsuKPkhrRokotnS8-LlGkEYPmCq87yWT.pdf
112 DEBUG: Selecting page 11 from /content/drive/MyDrive/AWCA/Colab_notebooks/OCR/pDonovan/awca-ocr/temp/1aaGfc5xmJOHC4S

mupdf: expected trailer marker
mupdf: invalid key in dict
mupdf: invalid key in dict
mupdf: invalid key in dict
mupdf: invalid key in dict
mupdf: invalid key in dict
mupdf: invalid key in dict
mupdf: invalid key in dict
mupdf: invalid key in dict
mupdf: invalid key in dict


169 DEBUG: Selecting page 3 from /content/drive/MyDrive/AWCA/Colab_notebooks/OCR/pDonovan/awca-ocr/temp/1pXHU7U6m2QJfr2_sJGDEK0rz5zDBi1OF.pdf
170 DEBUG: Selecting page 7 from /content/drive/MyDrive/AWCA/Colab_notebooks/OCR/pDonovan/awca-ocr/temp/1Oqk5VjKpkpy6IqB7RoykfiOfDkdaRiLO.pdf
171 DEBUG: Selecting page 50 from /content/drive/MyDrive/AWCA/PDFtp/ane.pdf.share/By Periodical/Sumer/Sumer Separata/المجلد الخامس و العشرون، 1969 - الجزء 1 و 2/noormags-Nineveh_The_1968_-_1969_Campagn.pdf
172 DEBUG: Selecting page 5 from /content/drive/MyDrive/AWCA/Colab_notebooks/OCR/pDonovan/awca-ocr/temp/1ShUR4ok7l2utFBH8pXD7LfCnNgQVl5PN.pdf
173 DEBUG: Selecting page 4 from /content/drive/MyDrive/AWCA/Colab_notebooks/OCR/pDonovan/awca-ocr/temp/1nE9H50mtPDhNKkMrgNCbZr9YDXOuNYM1.pdf
174 DEBUG: Selecting page 0 from /content/drive/MyDrive/AWCA/PDFtp/ane.pdf.share/By Topic (or field)/Dissertations/Studies in ancient Anatolian language and culture (Turkey) by Taylor, Patrick John.pdf
175 DEBUG: Selecting pag

...

mupdf: invalid key in dict
mupdf: invalid key in dict
mupdf: invalid key in dict
mupdf: invalid key in dict
mupdf: invalid key in dict
mupdf: invalid key in dict
mupdf: invalid key in dict
mupdf: invalid key in dict
mupdf: invalid key in dict


1327 DEBUG: Selecting page 0 from /content/drive/MyDrive/AWCA/Colab_notebooks/OCR/pDonovan/awca-ocr/temp/1AvQfRFlwXZFo0Xds7ci22AbPByqp-j11.pdf
1328 DEBUG: Selecting page 206 from /content/drive/MyDrive/AWCA/Colab_notebooks/OCR/pDonovan/awca-ocr/temp/17IxoBNxwjFmEAYY2ii_lhNJS4E6jCyRH.pdf
1329 DEBUG: Selecting page 1 from /content/drive/MyDrive/AWCA/PDFtp/ane.pdf.share/By Author (or editor)/L/Leemans, W.F. - The Trade Relations of Babylonia and the Question of Relations with Egypt in the OB Period. JESHO 3, 1960, pp. 21-37.pdf
1330 DEBUG: Selecting page 1 from /content/drive/MyDrive/AWCA/Colab_notebooks/OCR/pDonovan/awca-ocr/temp/1OfCSxawZps-mU6Bmr5jeUGjgreQ4mZEC.pdf
1331 DEBUG: Selecting page 16 from /content/drive/MyDrive/AWCA/Colab_notebooks/OCR/pDonovan/awca-ocr/temp/1kKJ2VzF2Yrh5-i-Txv3t_-ayAkA-D_CU.pdf
1332 DEBUG: Selecting page 3 from /content/drive/MyDrive/AWCA/Colab_notebooks/OCR/pDonovan/awca-ocr/temp/1r3QNGjeZ24zxiZH01W1MsLLbdlFkfSP0.pdf
1333 DEBUG: Selecting page 10 from /con

mupdf: invalid key in dict
mupdf: invalid key in dict
mupdf: invalid key in dict
mupdf: invalid key in dict
mupdf: invalid key in dict
mupdf: invalid key in dict
mupdf: invalid key in dict
mupdf: invalid key in dict
mupdf: invalid key in dict


1513 DEBUG: Selecting page 0 from /content/drive/MyDrive/AWCA/Colab_notebooks/OCR/pDonovan/awca-ocr/temp/1yBTQF9oiuNHuF2qdZ45wpTsBIFy57Qal.pdf
1514 DEBUG: Selecting page 17 from /content/drive/MyDrive/AWCA/Colab_notebooks/OCR/pDonovan/awca-ocr/temp/1uzguPi8xnS1GcwlFJTG5dYPE3_zad3Ch.pdf
1515 DEBUG: Selecting page 0 from /content/drive/MyDrive/AWCA/Colab_notebooks/OCR/pDonovan/awca-ocr/temp/1BBXISZYhcOlYRdJVkZGRZxi5OB1mLOyT.pdf
1516 DEBUG: Selecting page 14 from /content/drive/MyDrive/AWCA/Colab_notebooks/OCR/pDonovan/awca-ocr/temp/1T6ZVAqKUBgDOwhxuvl3YjBWEZOdUDbI6.pdf
1517 DEBUG: Selecting page 17 from /content/drive/MyDrive/AWCA/PDFtp/ane.pdf.share/By Topic (or field)/Area Studies/Egypt/Posener 1951 - La litterature Egyptienne (RdE 6).PDF
1518 DEBUG: Selecting page 0 from /content/drive/MyDrive/AWCA/Colab_notebooks/OCR/pDonovan/awca-ocr/temp/1-yme5RI4K-61BWnvws-922djUANOBpZR.pdf
1519 DEBUG: Selecting page 2 from /content/drive/MyDrive/AWCA/Colab_notebooks/OCR/pDonovan/awca-ocr/temp/1UQ

['no objects found',
 'cannot tell in file',
 'invalid key in dict',
 'cannot tell in file',
 'PDF must have at least one page.']

In [52]:
not 0

True

In [ ]:
current = !pwd
current = current[0]
current

In [ ]:
%cd $WORKING_DIR
ids = !ls
len(ids)

In [ ]:
'0B9Ibqa26YXiRUlVrZkVLd2ZvQmM.pdf' in ids

In [ ]:
ids[:10]

In [ ]:
any('1dYAjO7U60dPHJ0sJKkRN8PoiXAwb5NQO'.lower() in id.lower() for id in ids)

In [ ]:
[id for id in ids if len(id) < 10]

In [ ]:
os.path.splitext('/content/drive/MyDrive/AWCA/Colab_notebooks/OCR/pDonovan/awca-ocr/temp/1dYAjO7U60dPHJ0sJKkRN8PoiXAwb5NQO.pdf')[1]

## 2. Build a Compressed Representative Document

The objective here is to take a stratified sample of the pages in the large representative document.

## 3. Evaluate an OCR System

Here, I define a process for measuring _time_ and _overall accuracy_ for an OCR system.